## Find top similiar entities

### Word2Vec

In [26]:
# 將事件分祠但保留實體
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# 將上述結果丟入 Word2Vec 建立以語料為範圍之 embedding space 用來找出最相似的詞彙
from gensim.models import word2vec
import gensim
import json
import numpy as np
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tqdm import tqdm


In [3]:
ckip_ws = WS("ckiptagger/data", disable_cuda=False)

/home/abao.yang@cathayholdings.com.tw/.conda/envs/afa_chatbot/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/abao.yang@cathayholdings.com.tw/.conda/envs/afa_chatbot/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2022-06-16 00:41:15.721178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 00:41:15.884830: I tensorflow/stream_executor

In [3]:
input_filepath = os.path.join(
    "model_data", "output", "entity_extraction-output-20220614.json"
)
with open(input_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

len(data)
data[0]

{'title': '計程車每輛補助5千元防疫加清消費用 7月一次性撥付 | 金融脈動 | 金融 | 經濟日報',
 'url': 'https://money.udn.com/money/story/5613/6381896?from=edn_newestlist_rank',
 'meta_keywords': '交通部疫情觀光',
 'article': '計程車每輛補助5千元防疫加清消費用 7月一次性撥付 | 金融脈動 | 金融 | 經濟日報為協助司機駕駛因應疫情高原期影響，交通部表示，行政院已同意延長從6月至年底的防疫物資費用補貼，並已規畫延續每日防疫補助以及新增一次性清消防護費用，提高每輛計程車補貼金額共5000元，交通部也已請公路總局盡速完成準備作業，於7月進行撥付。交通部表示，為協助計程車營運防疫所需，自110年5月起，每天補貼計程車防疫物資費用15元，每個月補貼26天，原本至今年5月底止，在行政院支持下將延長至今年年底。考量計程車每日均載運不特定乘客且空間小與乘客近，因此除延續每日防疫物資補貼6至12月底的2730元，再新增一次性因應現況疫情加強清消及防護費用2270元，每輛計程車共計5000元，將於7月以一次性撥款方式匯到計程車車主帳戶，經費由交通部防疫預算支出。 針對遊覽車和租賃小客車短租車輛，交通部也規畫階段性的汽燃費減徵支持振興措施。在遊覽車部分，以減免徵111年第3季汽燃費為實施措施，出車率維持7成以上者減半徵收，未達7成者免徵，平均每輛車可節省約4000元至1萬元不等。小客車租賃業部分，因長租車均有固定承租客户，故規畫針對短租車提供111年第3季汽燃費減半徵收方案，平均每輛車可節省約1200元至1800元。交通部說，因應國內疫情及依據行政院現階段以防疫及振興為主的政策，針對目前仍受影響的觀光、公路客運、計程車、遊覽車及小客車租賃等產業，在行政院支持下，尋求各種可運用的資源，已分別因應各產業規畫不同的方案措施，協助業者因應疫情的影響，並將持續與各產業同步做好的振興準備。交通部擬發給計程車業每人一次性5000元防疫物資補貼。記者余承翰／攝影',
 'publish_time': '2022-06-12 09:54:27',
 'media': 'MoneyUDN',
 'event_triplets': [{'event': ['公路總局'

In [5]:
# 將事件實體根據順序拼出句子當作語料庫
ws_sents = []
for article in tqdm(data):
    entities = []
    for type, ent_list in article["entity_extract"].items():
        entities += ent_list
    entities = np.unique(entities).tolist()
    word_weight = {chr: 2 for chr in entities}
    for event_set in article["event_triplets"]:
        event = event_set["event"]
        ws_sent = []
        for i in range(len(event)):
            if i == 1:
                _ws = [event[i]]
            else:
                _ws = ckip_ws(
                    [event[i]], 
                    coerce_dictionary=construct_dictionary(word_weight)
                )[0]
            ws_sent += _ws
        ws_sents.append(ws_sent)

print(len(ws_sents))
ws_sents[:10]


100%|██████████| 244/244 [05:07<00:00,  1.26s/it]

8772


[['公路總局', '補助', '防疫', '加', '清', '消費用'],
 ['司機', '因應', '疫情', '高原期', '影響'],
 ['交通部',
  '表示',
  '行政院',
  '已',
  '同意',
  '延長',
  '從',
  '6月至年底',
  '防疫物資費用補貼',
  '並',
  '已',
  '規畫',
  '延續',
  '每日',
  '防疫補助',
  '以及',
  '新增',
  '一次性',
  '清消',
  '防護費用',
  '提高',
  '每',
  '輛',
  '計程車補貼金額',
  '共',
  '5000元'],
 ['行政院', '同意', '延長', '從', '6月至年底', '防疫物資費用補貼'],
 ['交通部', '請', '公路總局'],
 ['公路總局', '完成', '準備', '作業'],
 ['公路總局', '進行', '撥付'],
 ['交通部',
  '表示',
  '為',
  '協助',
  '計程車營運防疫',
  '需',
  '自',
  '110年5月',
  '起',
  '每',
  '天',
  '補貼計程車防疫物資費用',
  '15元',
  '每',
  '個',
  '月',
  '補貼',
  '26天',
  '原本',
  '至',
  '今年5月底止',
  '在',
  '行政院',
  '支持',
  '下',
  '將',
  '延長',
  '至',
  '今年年底'],
 ['計程車', '載運', '不', '特定', '乘客'],
 ['由', '交通部', '防疫', '預算', '支出', '經費']]

In [13]:
# Word2Vec train settings
seed = 888
sgs = [1] #[0, 1]
window_sizes = [10] #[1, 2, 3, 4, 5]
vector_sizes = [500] #[50, 100, 300, 500, 1000, 2000]
min_count = 2
workers = 8
epochs = [100] #[30, 50, 100]


# with open(f'external_output/word2vec_models/logging-{kw_col}.txt', 'w', encoding='utf-8') as f:
#     f.write(f'parameters: \nseed: {seed}\nwindow_size: {window_size}\nmin_count: {min_count}\nworkers: {workers}\n\n')

for epoch in tqdm(epochs):
    for sg in tqdm(sgs):
        for window_size in tqdm(window_sizes):
            for vector_size in tqdm(vector_sizes):

                model = word2vec.Word2Vec(
                    ws_sents,  # 語料
                    min_count=min_count,  # 詞彙的最小頻率，小於則從語料中排除
                    vector_size=vector_size,  # 詞向量維度
                    workers=workers,  # 使用多少 worker threads 訓練模型
                    epochs=epoch,  # iterations on this corpus
                    window=window_size,  # 多遠之內的詞要納入相似判斷範圍
                    sg=sg,  # 0=CBOW, 1=skip-gram
                    seed=seed,  # 向量初始化所需的隨機亂數生成
                    shrink_windows=True
                )
                
                # with open(f'external_output/word2vec_models/logging-{kw_col}.txt', 'a', encoding='utf-8') as f:
                #     f.write(f'===== epoch: {epoch} & sg: {sg} & vector_size: {vector_size} ===== \n')
                #     for test_word in ['信用卡','Line']:
                #         f.write(f'test word: {test_word}\n')
                #         res = model.wv.most_similar(test_word)
                #         for item in res:
                #             f.write(str(item))
                #             f.write('\n')
                #         f.write('\n')
                #     f.write('\n')

# model.save(f'external_output/word2vec/models/afa-{kw_col}-word2vec-epoch_{epoch}-sg_{sg}-vector_size_{vector_size}-window_size_{window_size}.model')


  0%|          | 0/1 [00:00<?, ?it/s]




100%|██████████| 1/1 [00:38<00:00, 38.63s/it]

100%|██████████| 1/1 [00:38<00:00, 38.64s/it]


In [18]:
model.wv.most_similar("交通部", topn=10)

[('5000元', 0.6660891175270081),
 ('一次性', 0.6519504189491272),
 ('車主', 0.641745924949646),
 ('計程車營運防疫', 0.6352728009223938),
 ('階段性', 0.6295223832130432),
 ('110年5月', 0.6273397207260132),
 ('減徵', 0.6254816651344299),
 ('計程車補貼金額', 0.6141300201416016),
 ('振興措施', 0.6136227250099182),
 ('6月至年底', 0.6130661368370056)]

### BERT Embeddings 
- 哈爾濱工業大學 中文全詞遮罩 chinese-roberta-wwm-ext-large


> 測試採用幾層 Embeddings 及不同 pooling strategy 對相似度的差異

In [67]:
import gc
import json
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import torch
torch.cuda.empty_cache()
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

from KGBuilder.config import *
from KGBuilder.data_utils import *

> 有部分可以針對把相似詞合併，但有一大群模稜兩可的實體被放到一起，再針對最大群進行分群


In [3]:
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
model = BertModel.from_pretrained(
    "hfl/chinese-roberta-wwm-ext-large",
    output_hidden_states = True
)
device = torch.device("cuda")
model = nn.DataParallel(model, device_ids=[0, 1])
model.to(device)
model.eval()


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DataParallel(
  (module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwi

In [4]:
input_filepath = os.path.join(
    "model_data", "output", "entity_extraction-output-20220614.json"
)
with open(input_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

len(data)
data[0]

{'title': '計程車每輛補助5千元防疫加清消費用 7月一次性撥付 | 金融脈動 | 金融 | 經濟日報',
 'url': 'https://money.udn.com/money/story/5613/6381896?from=edn_newestlist_rank',
 'meta_keywords': '交通部疫情觀光',
 'article': '計程車每輛補助5千元防疫加清消費用 7月一次性撥付 | 金融脈動 | 金融 | 經濟日報為協助司機駕駛因應疫情高原期影響，交通部表示，行政院已同意延長從6月至年底的防疫物資費用補貼，並已規畫延續每日防疫補助以及新增一次性清消防護費用，提高每輛計程車補貼金額共5000元，交通部也已請公路總局盡速完成準備作業，於7月進行撥付。交通部表示，為協助計程車營運防疫所需，自110年5月起，每天補貼計程車防疫物資費用15元，每個月補貼26天，原本至今年5月底止，在行政院支持下將延長至今年年底。考量計程車每日均載運不特定乘客且空間小與乘客近，因此除延續每日防疫物資補貼6至12月底的2730元，再新增一次性因應現況疫情加強清消及防護費用2270元，每輛計程車共計5000元，將於7月以一次性撥款方式匯到計程車車主帳戶，經費由交通部防疫預算支出。 針對遊覽車和租賃小客車短租車輛，交通部也規畫階段性的汽燃費減徵支持振興措施。在遊覽車部分，以減免徵111年第3季汽燃費為實施措施，出車率維持7成以上者減半徵收，未達7成者免徵，平均每輛車可節省約4000元至1萬元不等。小客車租賃業部分，因長租車均有固定承租客户，故規畫針對短租車提供111年第3季汽燃費減半徵收方案，平均每輛車可節省約1200元至1800元。交通部說，因應國內疫情及依據行政院現階段以防疫及振興為主的政策，針對目前仍受影響的觀光、公路客運、計程車、遊覽車及小客車租賃等產業，在行政院支持下，尋求各種可運用的資源，已分別因應各產業規畫不同的方案措施，協助業者因應疫情的影響，並將持續與各產業同步做好的振興準備。交通部擬發給計程車業每人一次性5000元防疫物資補貼。記者余承翰／攝影',
 'publish_time': '2022-06-12 09:54:27',
 'media': 'MoneyUDN',
 'event_triplets': [{'event': ['公路總局'

> ### 將實體區間標出後跑詞彙本身的 K-Means 分群
1. 取出完整事件句子及實體位置，轉換為 BERT Word Embeddings
2. 針對每個實體的 Word Embeddings 進行分群（K-Means 使用的也是 cosine simliarity）
   1. 先不去除所有句子統計下來的重複實體 => 因為上下文不同時的意思可能不同，如果將不同句子內的實體 embedding 先 mean 或 sum 處理，不確定會不會缺失原有語意；可能保留句子本身再比對較佳
     - 效果不太好，重複實體在不同句子的向量不同，但因為接近優先被聚到一群；其他相似詞的 threshold 變得很難調整
   2. 先將重複實體全部平均到一起，最後結果不同在哪句，只看實體本身

#### 取出完整事件句子及實體位置

In [154]:
# 紀錄每個句子之實體位置
for article in tqdm(data):
    for event_set in article["event_triplets"]:
        event_verb = event_set["event"][1]
        event = "".join(event_set["event"])
        entities = {
            event_verb: {
                "type": "VERB", 
                "position": re.search(event_verb, event).span()
            }
        }
        for type, ent_list in event_set["entity_extract"].items():
            ent_dict_list = []
            for ent in ent_list:
                ent_dict = {ent: {"type": type}}
                if re.search("\+|\|", ent):
                    re_ent = list(ent)
                    insert_num = 0
                    for span in re.finditer("\+|\|", ent):
                        icon_idx = span.span()
                        re_ent.insert(icon_idx[0]+insert_num, "\\")
                        insert_num += 1
                    re_ent = "".join(re_ent)
                else:
                    re_ent = ent
                ent_dict[ent]["position"] = re.search(re_ent, event).span()
                assert ent_dict[ent]["position"][0] != ent_dict[ent]["position"][1]
                entities.update(ent_dict)

        event_set["entity_extract_posi"] = entities


100%|██████████| 244/244 [00:00<00:00, 732.67it/s]


In [155]:
data[0]["event_triplets"]

[{'event': ['公路總局', '補助', '防疫加清消費用'],
  'entity_extract': {'ORG': ['公路總局'], 'OTH': ['防疫']},
  'entity_extract_posi': {'補助': {'type': 'VERB', 'position': (4, 6)},
   '公路總局': {'type': 'ORG', 'position': (0, 4)},
   '防疫': {'type': 'OTH', 'position': (6, 8)}}},
 {'event': ['司機', '因應', '疫情高原期影響'],
  'entity_extract': {'OTH': ['疫情']},
  'entity_extract_posi': {'因應': {'type': 'VERB', 'position': (2, 4)},
   '疫情': {'type': 'OTH', 'position': (4, 6)}}},
 {'event': ['交通部',
   '表示',
   '行政院已同意延長從6月至年底防疫物資費用補貼並已規畫延續每日防疫補助以及新增一次性清消防護費用提高每輛計程車補貼金額共5000元'],
  'entity_extract': {'ORG': ['行政院'],
   'MONEY': ['5000元'],
   'DATE': ['6月至年底'],
   'CARDINAL': ['一次性'],
   'OTH': ['防疫', '防疫物資費用補貼', '防疫補助', '防護費用', '計程車補貼金額'],
   'PRODUCT': ['計程車']},
  'entity_extract_posi': {'表示': {'type': 'VERB', 'position': (3, 5)},
   '行政院': {'type': 'ORG', 'position': (5, 8)},
   '5000元': {'type': 'MONEY', 'position': (64, 69)},
   '6月至年底': {'type': 'DATE', 'position': (14, 19)},
   '一次性': {'type': 'CARDINAL', 'position':

In [156]:
all_event_sents = []
for article in data:
    for event_item in article["event_triplets"]:
        all_event_sents.append({
            "event_text": "".join(event_item["event"]),
            "event_positions": event_item["entity_extract_posi"]
        })

max_length = max([len(sent["event_text"]) for sent in all_event_sents])
max_length

223

In [157]:
class SentenceDataset(torch.utils.data.Dataset):
    # 讀取資料、參數初始化
    def __init__(self, sentences, tokenizer, max_length):
        self.sentences = sentences
        self.len = len(self.sentences)
        self.max_length = max_length
        self.tokenizer = tokenizer

    # 回傳一筆訓練/測試資料
    def __getitem__(self, idx):
        text = self.sentences[idx]["event_text"]
        entities = self.sentences[idx]["event_positions"]
        tokens = self.tokenizer(text, max_length=self.max_length, padding="max_length", truncation=True)
        tokens_tensor = torch.tensor(tokens["input_ids"])
        segments_tensors = torch.tensor(tokens["token_type_ids"])
        masks_tensors = torch.tensor(tokens["attention_mask"])
        return (tokens_tensor, segments_tensors, masks_tensors, entities)

    def __len__(self):
        return self.len


def transform(encoded_layers):
    token_embeddings = torch.stack(encoded_layers, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token, dim=0)
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum


In [158]:
dataset = SentenceDataset(sentences=all_event_sents, tokenizer=tokenizer, max_length=max_length)
dataloader = DataLoader(dataset, batch_size=1)

In [175]:
all_entities = {}
for _data in tqdm(dataloader):
    if next(model.parameters()).is_cuda:

        torch.cuda.empty_cache()

        _data, entities = [t.to("cuda") for t in _data[:3] if t is not None], _data[3]
        tokens_tensor, segments_tensors, masks_tensors = _data
        _output = model(
            input_ids=tokens_tensor,
            token_type_ids=segments_tensors,
            attention_mask=masks_tensors
        )
        encoded_layers = _output[2]  # sentence vector
        # formatting the vector
        token_vec_sum = transform(encoded_layers=encoded_layers)
        # mapping the word vector by token vecor stack then sum up 
        for ent, values in entities.items():
            values["type"] = values["type"][0]
            # get entity position in the sentence
            start, end = values["position"][0].numpy()[0], values["position"][1].numpy()[0]
            values["position"] = (start, end)
            # compute word vector and copy to local cpu environment, to prevent CUDA OOM
            wv = torch.sum(torch.stack(token_vec_sum[start:end]), dim=0).cpu().detach().numpy()

            # if entity repeated, do torch.mean then add to all_entities(dict)
            if ent in all_entities:
                wv = np.mean([all_entities[ent]["wv"], wv], axis=0)
            all_entities[ent] = {"type": values["type"], "wv": wv}

        del entities
        gc.collect()


100%|██████████| 8772/8772 [43:10<00:00,  3.39it/s]


###

#### Entity embeddings 分群
- 沒有分 type
  - 幾乎自成一群 很分散
- 區分不同 type 跑分群
  - 能夠找到相似詞了，但遇到比較通用的詞彙（例如工會、市議會），及剛好在這個範圍中沒有多種講法的詞彙（例如：2353、台積電、台達）會在可以找到幾組相似詞匯的 threshold 下被集中到一大群
- 不管誰和誰一群，只找空間中最相似者


In [357]:
types

['OTH',
 'NORP',
 'LAW',
 'WORK_OF_ART',
 'TIME',
 'ORDINAL',
 'ORG',
 'LOC',
 'CARDINAL',
 'VERB',
 'PRODUCT',
 'ART',
 'PERSON',
 'PERCENT',
 'FAC',
 'MONEY',
 'QUANTITY',
 'EVENT',
 'GPE',
 'LANGUAGE',
 'DATE']

In [213]:
types = list(set([values["type"] for ent, values in all_entities.items()]))
all_entity_vectors = {}
for ent, values in tqdm(all_entities.items()):
    if values["type"] in all_entity_vectors:
        all_entity_vectors[values["type"]].append(values["wv"].tolist())
        all_entity_vectors[values["type"]+"-name"].append(ent)
    else:
        all_entity_vectors[values["type"]] = [values["wv"].tolist()]
        all_entity_vectors[values["type"]+"-name"] = [ent]

for type in all_entity_vectors:
    all_entity_vectors[type] = np.array(all_entity_vectors[type])

print(types)
all_entity_vectors["ORG"].shape

100%|██████████| 5230/5230 [00:00<00:00, 29675.89it/s]


(690, 1024)

In [312]:
from collections import Counter

from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import manifold

from scipy.cluster.hierarchy import single, ward, fcluster
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# using sklearn
clusterer = AgglomerativeClustering(n_clusters=None, affinity="manhattan", linkage="single", distance_threshold=3)
clusterer.fit(all_entity_vectors)
Counter(clusterer.labels_)


In [ ]:
# using kmeans
clusterer = KMeans(...)
clusterer.fit(all_entity_vectors)
Counter(clusterer.labels_)


In [364]:
# using scipy
_vectors = all_entity_vectors["EVENT"]
_ents = all_entity_vectors["EVENT-name"]

distance_Y = pdist(_vectors, 'euclidean')
linkage_Z = single(distance_Y)


In [365]:
# find most simliar 
distance_matrix_Y = squareform(distance_Y)

for query_eid in range(distance_matrix_Y.shape[0]):
    _pool = distance_matrix_Y[query_eid, :]
    nearest_eid = np.argpartition(_pool, 2)[1]
    print(_ents[query_eid], _ents[nearest_eid], _pool[nearest_eid])


南北戰爭 北戰爭 480.80251561789987
美國南北戰爭 南北戰爭 1020.3843476979752
對話會 座談會 969.5389822597508
九二 二戰 812.4386289167896
事件 會議 781.7473317594171
唐山黑社會團夥 黑道團伙事件 1348.100737310031
俄烏戰爭 烏俄戰爭 721.2171839820406
會議 集會 551.9291054845628
集會 會議 551.9291054845628
聯誼會 集會 861.0308424045578
台日聯誼會 聯誼會 1019.4234739834417
燒烤店群毆女子案 離譜毆女事件 1903.1026226708648
烏克蘭戰爭 俄烏戰爭 1024.779129600513
部長會議 集會 902.4373769175987
放映會 集會 1013.3116176163201
城市博覽會 畢業典禮 1174.5196122027326
專項行動 部長會議 1069.3176372291684
雷霆風暴 集會 1272.3544401926586
專項行動動員部署大會 專項行動 2352.9247829799815
歐洲國際人力潛艇競賽 國際勞工大會 2071.579895366774
票選活動 畢業典禮 984.6216136250993
理監事會 理監事會議 643.413230367707
第2季理監事會 理監事會議 1151.6225575052097
選舉 集會 699.6396029675989
安全會議 部長會議 1126.8177383297716
北戰爭 南北戰爭 480.80251561789987
行動集會 集會 1118.0198257434502
遊行集會 集會 1079.8727186764916
日經 會議 619.4098983583347
電影交響音樂 電影交響音樂會 509.1367063523519
電影交響音樂會 電影交響音樂 509.1367063523519
記者會 集會 902.6343532342445
畢業典禮 會議 914.0039439396606
校區畢業典禮 畢業典禮 1289.0637787217436
亞洲安全會議 安全會議 1262.6512490909056
座談會 

In [ ]:
# adjust dendrogram threshold
cluster_label = fcluster(linkage_Z, 500, criterion="distance")
cluster_dict = dict(Counter(cluster_label))
cluster_dict = {k: v for k, v in sorted(cluster_dict.items(), key=lambda item: item[1], reverse=True)}
cluster_dict

In [342]:
entity_agglomer = {}
for entity_id, cluster_id in tqdm(enumerate(cluster_label)):
    if cluster_id not in entity_agglomer:
        entity_agglomer[cluster_id] = []

    entity_agglomer[cluster_id].append(_ents[entity_id])

entity_agglomer = {k: v for k, v in sorted(entity_agglomer.items(), key=lambda item: len(item[1]), reverse=True)}

265it [00:00, 592478.98it/s]


In [343]:
for i, cluster in entity_agglomer.items():
    print("Cluster ", i, " ", len(cluster))
    print(cluster)
    print("")

Cluster  78   22
['中方', '道瓊', '中央', '企業', '央行', '中影', '美軍', '美中', '中共', '共軍', '共機', '共艦', '工會', '公司', '中油', '華信', '央視', '美銀', '車廠', '中企', '路透', '國安']

Cluster  65   2
['行政院', '政院']

Cluster  77   2
['俄軍', '俄砲']

Cluster  74   2
['歐盟', '白紙']

Cluster  72   2
['立院', '立委']

Cluster  142   1
['交通部']

Cluster  188   1
['道奇隊']

Cluster  152   1
['陸戰隊']

Cluster  62   1
['民進黨']

Cluster  54   1
['中國軍隊']

Cluster  147   1
['國防部長']

Cluster  1   1
['大陸衛健委']

Cluster  2   1
['衛健委']

Cluster  209   1
['中國代表團']

Cluster  116   1
['外交']

Cluster  11   1
['中央氣象局']

Cluster  130   1
['亞馬遜']

Cluster  223   1
['環球報']

Cluster  224   1
['Azot']

Cluster  90   1
['路透社']

Cluster  86   1
['媒體']

Cluster  33   1
['外交部']

Cluster  129   1
['台積電']

Cluster  175   1
['內政部']

Cluster  124   1
['當局']

Cluster  160   1
['Fed']

Cluster  68   1
['聯準會']

Cluster  104   1
['政府']

Cluster  69   1
['聯準']

Cluster  229   1
['Beta']

Cluster  213   1
['高貝他']

Cluster  190   1
['美國聯準會']

Cluster  165   1
['WTO']

Clust

In [275]:
_vectors = np.array([all_entity_vectors["ORG"][i].tolist() for i, cid in enumerate(cluster_label) if cid == 73])
_ents = np.array([all_entity_vectors["ORG-name"][i].tolist() for i, cid in enumerate(cluster_label) if cid == 73])
_vectors.shape

(265, 1024)

### FAISS
- FB 開發的最相似向量搜尋工具
- 雖然速度很快，但不確定 `index.search` 的結果為什麼是維度 `d` 變成最近鄰居個數（不是應該取樣本最近前5嗎？） 沒確認以前先不採用

In [186]:
import faiss


In [ ]:
ngpus = faiss.get_num_gpus()
print("number of GPUs:", ngpus)

d = all_entity_vectors.shape[1]
query = 
cpu_index = faiss.IndexFlatL2(d)
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
gpu_index.add(all_entity_vectors)  # add vectors to the index
print(gpu_index.ntotal)

k = 3  # get 3 nearest neighbors
D, I = gpu_index.search(xq, k)


In [189]:
all_entity_vectors.shape

(5230, 1024)